# Instruction tuning

In [ ]:
!pip install transformers
!pip install torch
!pip install accelerate
!pip install pyarrow
!pip install datasets

Today, we are breaking up the pipeline function from transformers that we have used previously. One of things the pipeline was doing behind the curtain was tokenising the text, but we can just as easily do that in a separate step.

Huggingface lets us initialize our model and tokenizer with the .from_pretrained() method, which will ensure that:
- we get a tokenizer that corresponds to the model architecture we want to use, and
- we download the vocabulary used when pretraining this specific checkpoint

In [32]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, max_length=250)



Now let's try to tokenise some text!

In [ ]:
input_text = "My name is "

tokenized_text = tokenizer(input_text, return_tensors="pt")
tokenized_text 

{'input_ids': tensor([[499, 564,  19,   3,   1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

The output is a dictionary, the first part of which are the input_ids are the IDs of the tokens in the vocabulary. We can check this by decoding the IDs back into words.

In [39]:
tokenizer.decode([0])

'<pad>'

The second part is the attention mask, which is a binary mask that tells the model which tokens to pay attention to and which to ignore (remember the causal vs fully-visible attention mask?). This is useful when we have padded our input to be the same length as the longest sequence in the batch, and we want the model to ignore the padding tokens.

Batched inputs are often different lengths, so they can’t be converted to fixed-size tensors. Padding and truncation are strategies for dealing with this problem, to create rectangular tensors from batches of varying lengths. Padding adds a special padding token to ensure shorter sequences will have the same length as either the longest sequence in a batch or the maximum length accepted by the model.

Try to add padding="max_length" to the tokenizer and see what happens. (You can find all the different possible values for the argument in the documentation: https://huggingface.co/transformers/main_classes/tokenizer.html)

In [41]:
tokenizer(input_text, return_tensors="pt", padding="max_length", truncation=True)

{'input_ids': tensor([[499, 564,  19,   3,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,

Truncation works in the other direction by truncating long sequences to the maximum length the model can accept. Try to insert a long sentence and add the truncation argument.

In most cases, padding your batch to the length of the longest sequence and truncating to the maximum length a model can accept works pretty well.

Now, let's move on to another part of the pipeline, which corresponds to the .generate() method. This method takes the token IDs and generates the next token IDs. We can do this in a separate step as well.


In [42]:
output = model.generate(tokenized_text["input_ids"])
output

/home/ucloud/.local/lib/python3.12/site-packages/transformers/generation/utils.py:1493: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


tensor([[  0,   3,   9,   3,   7,   9, 967,   1]])

We then only need to decode the IDs back into words to get the generated text.

In [43]:
tokenizer.decode(output[0])

'<pad> a sailor</s>'

Now, let's try and use the GPU for this task! We can do this by moving the model and the inputs to the GPU using the .to() method. 

In [44]:
model = model.to("cuda")
model.generate(tokenizer(input_text, return_tensors="pt").to("cuda")["input_ids"])

tensor([[  0,   3,   9,   3,   7,   9, 967,   1]], device='cuda:0')

We see that the device used is now cuda (the GPU) and the processing time is way faster!

Task
- Make your own function that works like the pipeline, but using the tokenization and generation steps we just saw

In [45]:
def your_pipeline_function(input_text):
    output = model.generate(tokenizer(input_text, return_tensors="pt").to("cuda")["input_ids"])
    return tokenizer.decode(output[0])

In [46]:
your_pipeline_function("hello")

'<pad> hello i am joking</s>'

## Machine translation

This week, we will attempt machine translation.

The dataset is the [OPUS-100](https://huggingface.co/datasets/Helsinki-NLP/opus-100) which contains translation pairs from over 100 languages. I chose the Danish and English translation pairs because that makes it easier for me to evaluate the quality of the translations, but feel very free to choose a different language pair if you prefer. You can see the different language pairs available in the "Subset" part of the dataset viewer in the link above.

We'll use huggingface's datasets library to load the dataset.

In [47]:
from datasets import load_dataset

ds = load_dataset("Helsinki-NLP/opus-100", "da-en", split='train[:1%]')

In [48]:
ds

Dataset({
    features: ['translation'],
    num_rows: 10000
})

In [49]:
ds["translation"]

[{'da': 'På Det Blandede EØS-Udvalgs vegne',
  'en': 'For the EEA Joint Committee'},
 {'da': 'metal, der indeholder mindst 99,9 vægtprocent bly, forudsat ingen anden bestanddel indgår i mængder, der overstiger de i nedenstående skema anførte grænseværdier:',
  'en': 'Metal containing by weight at least 99,9 % of lead, provided that the content by weight of any other element does not exceed the limit specified in the following table:'},
 {'da': 'Tænk.', 'en': 'Think.'},
 {'da': 'Beth...', 'en': 'Beth...'},
 {'da': 'Vort projekt "Menneskelig dvale" gør det muligt at holde vores bedste mænd nedfrosset i deres bedste tilstand, for at bruge dem efter behov.',
  'en': 'With the Human Hibernation Project, we will be able to save our best men... frozen in their prime, for use when they are needed most.'},
 {'da': '- Men hun siger nej.', 'en': "But she's turned him down."},
 {'da': 'RÅDET FOR DEN EUROPÆISKE UNION HAR —',
  'en': 'THE COUNCIL OF THE EUROPEAN UNION,'},
 {'da': 'Anhold mig.', 'en'

The translation pairs are nested in the translation column, so we need to flatten the dataset to get the source and target language in separate columns.

In [50]:
def unpack_cols(row):
    row["en"] = row["translation"]["en"]
    row["da"] = row["translation"]["da"]
    return row

train = ds.map(unpack_cols, remove_columns=["translation"])
train

Dataset({
    features: ['en', 'da'],
    num_rows: 10000
})

In [51]:
train[150]

{'en': 'That looks very painful, Viktor.',
 'da': 'Det ser smertefuldt ud, Viktor.'}

Now, try to pick a few sentences and see how well the model translates them without any additional help - just giving the source language sentence as input and letting the model generate the translation.

try to pick a few sentences and see how well the model can translate out of the box.

In [52]:
your_pipeline_function(train[150]['en'])


'<pad> That looks very painful, Viktor.</s>'

- Did it work? If not, why?

If zero shot prompting didn't work,

- try one shot and
- few shot prompting, to see if providing context helps the model to generate better translations.

In [ ]:
your_pipeline_function(f"English: {train[150]['en']} Danish:")

'<pad> Viktor oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss oss os'

Now, let's try instruction  tuning the model to hopefully get a better result!

The datasets library has a nice map method that we can use to apply a function to all the examples in the dataset. The map method can take a custom function, so we just need to write a function that prepares our data for the model.

Write preprocessing function that takes in a batch of the dataset and
- defines an instruction and appends it to the input text
- creates a list of all input texts (hint: you can use a loop or list comprehension)
- creates a new column in the dataset called "input_ids" that contains the token ids of the input text (hint: you can use the tokenizer on the list of input texts)
- creates a list of all output texts
- creates a new column in the dataset called "labels" that contains the token ids of the output text
- returns the batch

If you need a bit of help, I've started the first three steps for you.

In [58]:
def preprocessing_func(batch): 
    input_texts = ["""English: '""" + row + """' Danish: '""" for row in batch['en']]
    batch["input_ids"] = tokenizer(input_texts, padding="max_length", truncation=True, return_tensors="pt").to("cuda").input_ids
    labels = [row for row in batch["da"]]
    batch["labels"] = tokenizer(labels, padding="max_length", truncation=True, return_tensors="pt").to("cuda").input_ids
    return batch

In [59]:
tokenized_train = train.map(preprocessing_func, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [60]:
tokenized_train

Dataset({
    features: ['en', 'da', 'input_ids', 'labels'],
    num_rows: 10000
})

In [61]:
tokenized_train = tokenized_train.remove_columns(["en", "da"])
tokenized_train

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 10000
})

Now the data is ready for the model, so we can fine-tune it!

In [63]:
tokenized_train[0]["labels"]

[276,
 2,
 374,
 17,
 272,
 16349,
 15,
 262,
 2,
 134,
 18,
 1265,
 26,
 2165,
 122,
 7,
 3,
 162,
 15141,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


We then want to initalize a Trainer class.

To do this, we have to defined the TrainingArguments, which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional.

I have changed a few parameters, like the learning rate and weight decay, as well as setting the max number of steps (so it doesn't run for a very long time) and the logging steps (so we get updated more frequently on the loss) and the batch size (also for speed). If you want to play around with changing other parameters, you can find the full list of arguments in the documentation (https://huggingface.co/docs/transformers/en/main_classes/trainer).

In [64]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(output_dir="./flan-t5-small-da-en",
   per_device_train_batch_size=4,
   learning_rate=1e-3,
   weight_decay=0.01,
   max_steps=3000,
   logging_steps=200,
)

trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
)

max_steps is given, it will override any value given in num_train_epochs


Now we're ready to train! Buckle up, this will probably take a bit of time...

In [65]:
trainer.train()

Step,Training Loss
200,1.315100
400,0.165300
600,0.166100
800,0.147600
1000,0.141500
1200,0.144600
1400,0.140700
1600,0.131000
1800,0.130100
2000,0.129900


/home/ucloud/.local/lib/python3.12/site-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 250}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3000, training_loss=0.21638838068644206, metrics={'train_runtime': 668.987, 'train_samples_per_second': 17.938, 'train_steps_per_second': 4.484, 'total_flos': 2230686056448000.0, 'train_loss': 0.21638838068644206, 'epoch': 1.2})

We can now save our model and load it in as a pretrained model!

In [66]:
trainer.save_model("instruct-model")

In [67]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("instruct-model").to("cuda")

You can adjust your pipeline to incorporate the new instruct_model!

In [68]:
def your_pipeline_function(input_text):
    output = instruct_model.generate(tokenizer(input_text, return_tensors="pt").to("cuda")["input_ids"])
    return tokenizer.decode(output[0])

In [71]:
your_pipeline_function(f"Question: How many meters are there in a kilometer? Answer: ")

'<pad> Hvor m<unk>nedt meter er der i en kilometer?</s>'

Try to test the finetuned model on the examples from before.

- Does it perform better than before? Why/why not?
- What happens if you instruct the model to perform a different task, like summarisation or reasoning? Does the performance gain transfer? Why/why not?
- If you wanted to instruction tune a model to be able to solve a wide variety of tasks (like chatpgt), what kind of training data would you need?
- How would you produce that kind of data and what are the possible limitations?